In [1]:
!pip install pandas numpy torch matplotlib hdbscan umap-learn scikit-learn sentence-transformers
!pip install bertopic==0.16.2
!pip install demoji
!pip install advertools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.3/252.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.2/254.2 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6

In [2]:
!pip install -qq accelerate
!pip install -qq datasets
!pip install -qq trl
!pip install -qq transformers
!pip install -qq bitsandbytes

In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
import demoji
from statistics import mean
from torch import cuda
from transformers import (AutoModelForCausalLM,
                          AutoModelForSeq2SeqLM,
                          AutoTokenizer,
                          GenerationConfig,
                          BitsAndBytesConfig,
                          pipeline,
                          logging)
from bertopic import BERTopic
from statistics import mean
import torch
import matplotlib.pyplot as plt
import hdbscan
import os
import io
import re
import string
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from umap import UMAP
from sklearn.metrics.pairwise import cosine_distances
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
import seaborn as sns
import advertools as adv
from bertopic.representation import TextGeneration
from bertopic.vectorizers import ClassTfidfTransformer
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/opt/conda/lib/python3.10/site-packages/scrapy/utils/_compression.py:18: ScrapyDeprecationWarning: You have brotlipy installed, and Scrapy will use it, but Scrapy support for brotlipy is deprecated and will stop working in a future version of Scrapy. brotlipy itself is deprecated, it has been superseded by brotlicffi. Please, uninstall brotlipy and install brotli or brotlicffi instead. brotlipy has the same import name as brotli, so keeping both installed is strongly discouraged.
  warn(


In [4]:
def remove_emojis(text):
    return demoji.replace(text, '')

def preprocess_text(text):
    # Remove retweets
    text = re.sub(r'^RT[\s]+', '', text, flags=re.IGNORECASE)
    # Remove usernames @
    text = re.sub(r'@[^\s]+', '', text)
    # Remove URLs
    text = re.sub(r'https\S+', '', text)
    url_words = ['url', 'URL', 'html', 'HTML', 'http', 'HTTP']
    for u in url_words:
        text = re.sub(u, '', text)
    # Remove numbers
    text = re.sub(r'\b\d+\b', '', text)
    text = remove_emojis(text)
    # Remove extra spaces that may have been introduced
    text = re.sub(r'\s+', ' ', text).strip()
    # Return preprocessed text
    return text

def load_dataset(input_file, stance, topic) -> pd.DataFrame:
    '''
    Each time we only deal with a single topic and stance.
    '''
    input_corpus = pd.read_csv(input_file)
    input_corpus = input_corpus[input_corpus["topic"] == topic]

    if stance == 1:
        input_corpus = input_corpus[input_corpus["stance"] == 1]
    else:
        input_corpus = input_corpus[input_corpus["stance"] == -1]

    input_corpus.reset_index(drop=True, inplace=True)

    # Debugging: Check if DataFrame is empty after filtering
    if input_corpus.empty:
        print("No data available for the specified topic and stance.")
        return pd.DataFrame()  # Return an empty DataFrame

    # Preprocess the arguments and store them in a new column
    preprocessed_data = [preprocess_text(arg) for arg in input_corpus['argument']]

    # Debugging: Check preprocessed data
    if not preprocessed_data:
        print("Preprocessed data is empty. Check the preprocessing function.")

    # Expand the tuples into separate columns
    input_corpus['preprocessed_arguments']= preprocessed_data

    # Debug output
    print("Loaded dataset:")
    print(input_corpus.head())
    print(f"Total records: {len(input_corpus)}")

    return input_corpus

In [5]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_hf = user_secrets.get_secret("HUGGINGFACE_TOKEN")
!huggingface-cli login --token $secret_hf

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
model_id = "ilsp/Meltemi-7B-Instruct-v1.5"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16)
# Llama 2 Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Llama 2 Model
instruct_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)
instruct_model.eval()

tokenizer_config.json:   0%|          | 0.00/3.65k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.97M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/503M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(61384, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNo

In [7]:
from datasets import Dataset

def create_huggingface_dataset(data_set, argument_column='arg'):
    # Step 1: Filter to keep only rows with label '1'
    filtered_data = data_set[data_set['label'] == 1]

    # Step 2: Group by 'key_point', 'topic', and 'stance' and count arguments
    # Keep only the first 4 arguments in each group
    grouped_data = filtered_data.groupby(['key_point', 'topic', 'stance'])[argument_column] \
        .apply(lambda x: '\n'.join([f"- {arg}" for arg in x[:4]])).reset_index()

    # Step 3: Count the number of arguments for each 'key_point' (if needed)
    argument_counts = filtered_data.groupby(['key_point', 'topic', 'stance']).size().reset_index(name='count')

    # Step 4: Merge the argument counts with grouped data (optional, can be removed if not needed)
    grouped_data = grouped_data.merge(argument_counts, on=['key_point', 'topic', 'stance'])

    # Step 5: Create a Hugging Face Dataset including 'key_point', 'topic', 'stance', and the formatted arguments
    hf_dataset = Dataset.from_pandas(grouped_data[['key_point', 'topic', 'stance', argument_column]])

    return hf_dataset

# Example Usage:
# Load your train and dev DataFrames
train_df = pd.read_csv('/kaggle/input/meltemi-data/train_dev_test_dataset/final_data/train_gr.csv')
dev_df = pd.read_csv('/kaggle/input/meltemi-data/train_dev_test_dataset/final_data/dev_human_gr.csv')

# Create Hugging Face datasets
train_hf_dataset = create_huggingface_dataset(train_df)
dev_hf_dataset = create_huggingface_dataset(dev_df)


def get_filtered_rows(hf_dataset):
    # Convert the Hugging Face Dataset to a pandas DataFrame for easier manipulation
    df = hf_dataset.to_pandas()

    # Step 1: Find topics that contain at least one 'for' (stance = 1) and one 'against' (stance = -1)
    qualifying_topics = []
    for topic in df['topic'].unique():
        topic_df = df[df['topic'] == topic]
        
        # Check if the topic has at least one positive and one negative stance
        has_positive = not topic_df[topic_df['stance'] == 1].empty
        has_negative = not topic_df[topic_df['stance'] == -1].empty
        
        if has_positive and has_negative:
            qualifying_topics.append(topic)
        
        # Stop once we have 4 qualifying topics
        if len(qualifying_topics) == 2:
            break

    # Step 2: Filter the DataFrame to include only the selected topics
    filtered_df = df[df['topic'].isin(qualifying_topics)]

    # Step 3: For each selected topic, get the first positive and first negative stance
    result_df = pd.DataFrame()  # Initialize an empty DataFrame to store results
    for topic in qualifying_topics:
        topic_df = filtered_df[filtered_df['topic'] == topic]
        
        # Get the first row with stance -1 and stance 1
        first_negative = topic_df[topic_df['stance'] == -1].head(1)
        first_positive = topic_df[topic_df['stance'] == 1].head(1)
        
        # Concatenate the results to the result DataFrame
        result_df = pd.concat([result_df, first_negative, first_positive])

    # Step 4: Convert the result back to a Hugging Face Dataset
    filtered_hf_dataset = Dataset.from_pandas(result_df.reset_index(drop=True))

    return filtered_hf_dataset

# Apply the function to get the filtered dataset
filtered_train_hf_dataset = get_filtered_rows(train_hf_dataset)


In [8]:
# Define the system message
system_message = {
    "role": "system",
    "content": "Είσαι το Μελτέμι, ένα γλωσσικό μοντέλο για την ελληνική γλώσσα. "
               "Είσαι ιδιαίτερα βοηθητικό προς την χρήστρια ή τον χρήστη και δίνεις σύντομες αλλά επαρκώς περιεκτικές απαντήσεις. "
               "Απάντα με προσοχή, ευγένεια, αμεροληψία, ειλικρίνεια και σεβασμό προς την χρήστρια ή τον χρήστη."
}

# Example template to structure demonstration examples
example_template = """Τα παρακάτω επιχειρήματα υποστηρίζουν ή αντικρούουν το θέμα. Συμπλήρωσε ένα συνοπτικό keypoint, καταγράφοντας την κεντρική ιδέα των επιχειρημάτων σε μία πρόταση.
[ARGUMENTS]"""

# Function to load demonstrations from the dataset as message entries
def load_demonstrations_as_messages(hf_dataset):
    # Convert Hugging Face dataset to pandas DataFrame for easy access
    df = hf_dataset.to_pandas()

    # Create list of messages, starting with the system message
    messages = [system_message]

    # Build demonstration examples, including both user prompts and assistant responses
    for _, row in df.iterrows():
        # Format arguments as a bullet-point list
        arguments_text = '\n'.join(f"{arg}" for arg in row['arg'].split('\n'))
        #topic_text=row['topic']
        # Construct the user and assistant messages for each demonstration
        example_user_text = example_template.replace("[ARGUMENTS]", arguments_text) #.replace("TOPIC",topic_text)
        example_assistant_text = row['key_point']
        
        # Append the user and assistant demonstration messages
        messages.append({"role": "user", "content": example_user_text})
        messages.append({"role": "assistant", "content": example_assistant_text})

    # Main prompt to be added as the final user message
    main_prompt = """Τα παρακάτω επιχειρήματα υποστηρίζουν ή αντικρούουν το θέμα. Συμπλήρωσε ένα συνοπτικό keypoint, καταγράφοντας την κεντρική ιδέα των επιχειρημάτων σε μία πρόταση.
[DOCUMENTS]"""
    messages.append({"role": "user", "content": main_prompt})

    return messages

# Apply the chat template
messages = load_demonstrations_as_messages(filtered_train_hf_dataset)
prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
print(prompt)

<|system|>
Είσαι το Μελτέμι, ένα γλωσσικό μοντέλο για την ελληνική γλώσσα. Είσαι ιδιαίτερα βοηθητικό προς την χρήστρια ή τον χρήστη και δίνεις σύντομες αλλά επαρκώς περιεκτικές απαντήσεις. Απάντα με προσοχή, ευγένεια, αμεροληψία, ειλικρίνεια και σεβασμό προς την χρήστρια ή τον χρήστη.</s>
<|user|>
Τα παρακάτω επιχειρήματα υποστηρίζουν ή αντικρούουν το θέμα. Συμπλήρωσε ένα συνοπτικό keypoint, καταγράφοντας την κεντρική ιδέα των επιχειρημάτων σε μία πρόταση.
- Το κλείσιμο του θα οδηγήσει μόνο σε περισσότερους τρομοκράτες που θα τρέχουν ελεύθεροι και έτσι θα θέσουν σε κίνδυνο την ασφάλειά μας.
- Η κράτηση τρομοκρατών στο Γκουαντάναμο εξασφαλίζει ότι δεν μπορούν να σχεδιάσουν μελλοντικές επιθέσεις εναντίον της Αμερικής.
- Η κράτηση τρομοκρατών στο Γκουαντάναμο τους κρατάει μακριά από το πεδίο της μάχης.
- Για λόγους ασφαλείας πρέπει να κρατήσουμε το στρατόπεδο ανοιχτό.</s>
<|assistant|>
Το κλείσιμο του στρατοπέδου κράτησης του Γκουαντάναμο θα βλάψει την εθνική ασφάλεια</s>
<|user|>
Τα παρα

In [9]:
# Our text generator
generator = pipeline(
    model=instruct_model,
    tokenizer=tokenizer,
    task='text-generation',
    max_new_tokens=100,
    num_beams=3,
    early_stopping=True,
    length_penalty= 2.0
)
meltemi_instruct = TextGeneration(generator, prompt=prompt)

In [10]:
# Topic Modeling Script for Jupyter Notebook
# --------------------------------------------------------------
n_neighbors = 15  # Set your desired value
n_components = 2  # Set your desired value
min_samples_fraction = 1.0  # Set your desired value (as a fraction)
selected_arguments_path = '/kaggle/input/meltemi-data/'
output_topic_data = './output_topic_data/'  # Output directory for topic data
output_arguments_data = './output_arguments_data/'  # Output directory for arguments data
stopwords = list(adv.stopwords['greek'])

# Set environment variable to avoid parallelism issues with tokenizers
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# Set device for torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def topic_modeling(df, output_prefix, topic, stance):
    cluster_size = int(len(df) / 50)
    if cluster_size < 3:
        cluster_size = 3

    print(f"Cluster size: {cluster_size}")

    min_samples = int(min_samples_fraction * cluster_size)
    if min_samples < 2:
        min_samples = 2

    print(f"Min samples: {min_samples}")

    text_list = df['preprocessed_arguments'].tolist()
    embedding_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2", trust_remote_code=True)
    embeddings = embedding_model.encode(text_list)
    umap_model = UMAP(random_state=42, n_neighbors=n_neighbors, n_components=n_components, min_dist=0.00, metric='cosine')
    hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=cluster_size,
                                    metric='euclidean',
                                    cluster_selection_method='leaf',
                                    min_samples=min_samples,
                                    prediction_data=True)
    vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords, lowercase=True)
    ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
    representation_model = {"Meltemi_Instruct": meltemi_instruct}
    topic_model = BERTopic(
        embedding_model=embedding_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=vectorizer_model,
        ctfidf_model=ctfidf_model,
        representation_model=representation_model,
        low_memory=True,
        nr_topics=10,
        verbose=True,
        calculate_probabilities=True,
        language="multilingual"
    )
    # Fit the model and transform the documents
    topics, probabilities = topic_model.fit_transform(documents=text_list, embeddings=embeddings)

    # Ensure output directory exists
    os.makedirs(output_topic_data, exist_ok=True)

    # Save topic information to CSV
    topic_info = topic_model.get_topic_info()

    # Add 'topic' and 'stance' columns to the topic information
    topic_info['topic'] = topic
    topic_info['stance'] = stance

    # Save the updated topic information to CSV
    output_file = os.path.join(output_topic_data, f"{output_prefix}_topic_info.csv")
    topic_info.to_csv(output_file, index=False)
    print(f"Topic information saved to {output_file}")

    meltemi_base_labels = [label[0][0].split("\n")[0] for label in topic_model.get_topics(full=True)["Meltemi_Instruct"].values()]
    topic_model.set_topic_labels(meltemi_base_labels)

    # Save topic visualization as HTML
    visualization = topic_model.visualize_documents(text_list, embeddings=embeddings,hide_annotations=True, hide_document_hover=False, custom_labels=True)
    visualization_output_file = os.path.join(output_topic_data, f"{output_prefix}_visualization.html")
    visualization.write_html(visualization_output_file)

    print(f"Topic visualization saved to {visualization_output_file}")
    


# Load the main CSV file that contains topics and stances
arguments_file_path = os.path.join(selected_arguments_path, 'arguments_human_translated_dev.csv')
arguments_df = pd.read_csv(arguments_file_path)

# Get unique combinations of topic and stance
unique_combinations = arguments_df[['topic', 'stance']].drop_duplicates()

# Iterate through each unique combination of topic and stance
for _, row in unique_combinations.iterrows():
    topic = row['topic']
    stance = row['stance']

    # Load dataset based on the topic and stance
    filtered_df = load_dataset(arguments_file_path, stance, topic)

    # Perform topic modeling and save results
    topic_modeling(filtered_df, output_prefix=f"{topic}_{stance}", topic=topic, stance=stance)

Loaded dataset:
    arg_id                                              topic  stance  \
0  arg_4_0  Θα πρέπει να καταργηθεί η μαθητική στολή στα σ...      -1   
1  arg_4_1  Θα πρέπει να καταργηθεί η μαθητική στολή στα σ...      -1   
2  arg_4_2  Θα πρέπει να καταργηθεί η μαθητική στολή στα σ...      -1   
3  arg_4_3  Θα πρέπει να καταργηθεί η μαθητική στολή στα σ...      -1   
4  arg_4_4  Θα πρέπει να καταργηθεί η μαθητική στολή στα σ...      -1   

                                            argument  \
0  το να φορούν οι μαθητές στολή μπορεί να μειώσε...   
1  είναι ένας καλός τρόπος να ενσταλλάξεις την πε...   
2  η αγορά μαθητικής στολής είναι μια πιο φθηνή λ...   
3  ένα από τα οφέλη της μαθητικής στολής είναι ότ...   
4  έρευνες δείχνουν ότι στα σχολεία εκείνα ανα το...   

                              preprocessed_arguments  
0  το να φορούν οι μαθητές στολή μπορεί να μειώσε...  
1  είναι ένας καλός τρόπος να ενσταλλάξεις την πε...  
2  η αγορά μαθητικής στολής είναι μια πιο φ

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

2024-11-20 17:58:27,886 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-20 17:58:28,055 - BERTopic - Dimensionality - Completed ✓
2024-11-20 17:58:28,056 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-20 17:58:28,066 - BERTopic - Cluster - Completed ✓
2024-11-20 17:58:28,067 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 13/13 [03:34<00:00, 16.53s/it]
2024-11-20 18:02:03,061 - BERTopic - Representation - Completed ✓
2024-11-20 18:02:03,062 - BERTopic - Topic reduction - Reducing number of topics
100%|██████████| 10/10 [02:54<00:00, 17.43s/it]
2024-11-20 18:04:57,458 - BERTopic - Topic reduction - Reduced number of topics from 13 to 10


Topic information saved to ./output_topic_data/Θα πρέπει να σταματήσει η θετική δράση_-1_topic_info.csv
Topic visualization saved to ./output_topic_data/Θα πρέπει να σταματήσει η θετική δράση_-1_visualization.html
Loaded dataset:
       arg_id                                   topic  stance  \
0  arg_15_108  Θα πρέπει να σταματήσει η θετική δράση       1   
1  arg_15_109  Θα πρέπει να σταματήσει η θετική δράση       1   
2  arg_15_110  Θα πρέπει να σταματήσει η θετική δράση       1   
3  arg_15_111  Θα πρέπει να σταματήσει η θετική δράση       1   
4  arg_15_112  Θα πρέπει να σταματήσει η θετική δράση       1   

                                            argument  \
0  η θετική δράση δημιουργεί μεγαλύτερη δυσαρέσκε...   
1  η θετική δράση μπορεί να θεωρηθεί διάκριση εις...   
2  η θετική δράση μπορεί να οδηγήσει ανθρώπους με...   
3  η θετική δράση δυσκολεύει τους προσοντούχους υ...   
4  η θετική δράση στερεί από τα πιο προσοντούχα ά...   

                              preprocessed

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2024-11-20 18:05:01,492 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-20 18:05:01,678 - BERTopic - Dimensionality - Completed ✓
2024-11-20 18:05:01,679 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-20 18:05:01,689 - BERTopic - Cluster - Completed ✓
2024-11-20 18:05:01,689 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 13/13 [03:07<00:00, 14.44s/it]
2024-11-20 18:08:09,500 - BERTopic - Representation - Completed ✓
2024-11-20 18:08:09,501 - BERTopic - Topic reduction - Reducing number of topics
100%|██████████| 10/10 [02:26<00:00, 14.65s/it]
2024-11-20 18:10:36,085 - BERTopic - Topic reduction - Reduced number of topics from 13 to 10


Topic information saved to ./output_topic_data/Θα πρέπει να σταματήσει η θετική δράση_1_topic_info.csv
Topic visualization saved to ./output_topic_data/Θα πρέπει να σταματήσει η θετική δράση_1_visualization.html


In [11]:
# Define the directory where your CSV files are located
csv_directory = './output_topic_data/'  # Replace with the path to your CSV files
merged_output_file = './4_shot_4_args_beam_3_final.csv'  # Path for the merged output file

# Create an empty list to hold DataFrames
dataframes = []

# Iterate over all files in the directory
for file in os.listdir(csv_directory):
    if file.endswith('.csv'):
        file_path = os.path.join(csv_directory, file)
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        # Append the DataFrame to the list
        dataframes.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
merged_df = pd.concat(dataframes, ignore_index=True)
merged_df.to_csv(merged_output_file, index=False)